In [63]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [64]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-20.43,100,57,1.80,RU,1698691020
1,1,margate,51.3813,1.3862,10.17,91,100,0.45,GB,1698691056
2,2,port-aux-francais,-49.3500,70.2167,2.83,78,100,15.49,TF,1698690936
3,3,isafjordur,66.0755,-23.1240,0.02,86,75,1.03,IS,1698690943
4,4,edinburgh of the seven seas,-37.0676,-12.3116,12.09,90,100,14.09,SH,1698690938


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    frame_width = 1000,
    frame_height = 600
)

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [74]:
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 24) & (city_data_df['Max Temp'] <= 28) & (city_data_df['Humidity'] <= 85) & (city_data_df['Wind Speed'] < 5)]
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,benoy,8.9833,16.3199,26.72,63,100,0.84,TD,1698691230
28,28,ratnagiri,16.9833,73.3000,26.02,64,9,1.51,IN,1698691232
37,37,rudbar,30.1500,62.6000,25.39,24,3,1.38,AF,1698691235
46,46,mandera,3.9366,41.8670,25.99,69,94,1.97,KE,1698691237
48,48,kabo,7.6994,18.6290,25.93,66,2,1.53,CF,1698691238
...,...,...,...,...,...,...,...,...,...,...
498,498,porbandar,21.6422,69.6093,27.50,56,0,2.88,IN,1698691346
502,502,waingapu,-9.6567,120.2641,25.14,84,54,2.32,ID,1698691130
507,507,inyonga,-6.7167,32.0667,26.46,36,100,1.66,TZ,1698691348
522,522,salalah,17.0151,54.0924,27.05,78,0,2.06,OM,1698691352


In [67]:
# use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
27,caleta de carquin,PE,-11.0925,-77.6267,76,
29,pushkar,IN,26.5000,74.5500,33,
30,adamstown,PN,-25.0660,-130.1015,76,
41,chingola,ZM,-12.5290,27.8838,57,
102,extrema,BR,-22.8547,-46.3183,85,
109,pisco,PE,-13.7000,-76.2167,64,
169,saint-pierre,RE,-21.3393,55.4781,78,
181,rusera,IN,25.7500,86.0333,54,
183,asind,IN,25.7333,74.3333,40,
190,nova sintra,CV,14.8667,-24.7167,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [75]:
# set the empty geo coordinates
lat = []
lng = []

# set the parameters for the type of search 
base_url = 'https://api.geoapify.com/v2/places'
categories = 'accommodation.hotel'
limit = 5
bias = f'proximity:{lng},{lat}'
radius = 10000
filters = f'circle:{lng},{lat},{radius}'

params = {'categories': categories,
          'limit': limit,
          'bias': bias, 
          'apiKey': geoapify_key, 
          'filter': filters
          }

In [69]:
# test the code to see the output

# response = requests.get(base_url, params=params).json()
# print(json.dumps(response, indent=4, sort_keys=True))

In [76]:


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
caleta de carquin - nearest hotel: Hotel Mi Choacan
pushkar - nearest hotel: Hotel Hill View
adamstown - nearest hotel: No hotel found
chingola - nearest hotel: Protea Hotel
extrema - nearest hotel: Hotel San Marco
pisco - nearest hotel: La Portada
saint-pierre - nearest hotel: Tropic Hotel
rusera - nearest hotel: No hotel found
asind - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
mersa matruh - nearest hotel: صقور
kakonko - nearest hotel: No hotel found
pueblo nuevo - nearest hotel: No hotel found
mufulira - nearest hotel: No hotel found
taghazout - nearest hotel: Riad Taghazout
argostolion - nearest hotel: Ainos Hotel
narasannapeta - nearest hotel: No hotel found
saint-joseph - nearest hotel: PALM Hotel & Spa
tsiombe - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
cova figueira - nearest hotel: No hotel found
cheria - nearest hotel: No hotel found
cotacachi - nearest hotel: Posada Munaylla
obo - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
27,caleta de carquin,PE,-11.0925,-77.6267,76,Hotel Mi Choacan
29,pushkar,IN,26.5000,74.5500,33,Hotel Hill View
30,adamstown,PN,-25.0660,-130.1015,76,No hotel found
41,chingola,ZM,-12.5290,27.8838,57,Protea Hotel
102,extrema,BR,-22.8547,-46.3183,85,Hotel San Marco
109,pisco,PE,-13.7000,-76.2167,64,La Portada
169,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
181,rusera,IN,25.7500,86.0333,54,No hotel found
183,asind,IN,25.7333,74.3333,40,No hotel found
190,nova sintra,CV,14.8667,-24.7167,63,Residência Ka Dencho


### Add the hotel name and the country as additional information in the hover message for each city in the map

In [78]:
# configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    frame_width = 1000,
    frame_height = 600,
    hover_cols = ['Hotel Name', 'Country']
)

# display the map plot
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

### Additional work
Since this is a vacation, let's remove the locations where there are no hotels 😉

In [79]:
cities_with_hotels = hotel_df[hotel_df['Hotel Name']!= 'No hotel found']


In [73]:
# configure the map plot
map_plot_3 = cities_with_hotels.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    frame_width = 1000,
    frame_height = 600,
    hover_cols = ['Hotel Name', 'Country']
)

# display the map plot
map_plot_3


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)